In [0]:
products=[("USA","productA",1000,2000),
          ("India","productB",500,50),
          ("UK","productC",700,100),
          ("Canada","productD",200,20)]
columns=["country","product","stock","sales"]
spark.createDataFrame(products,columns).write.format('delta').mode('overwrite').saveAsTable("silver_tb")

In [0]:
import pyspark.sql.functions as F
spark.read.format("delta").table("silver_tb").withColumn("SalesRate",F.col("Sales")/F.col("Stock"))\
    .drop("Stock").drop("Sales")\
    .write.format("delta").mode("overwrite").saveAsTable("Gold_tb")

In [0]:
%sql
--Select * from silver_tb;
Select * from Gold_tb;

country,product,SalesRate
Australia,productE,0.1
UK,productC,0.14285714285714285
USA,productA,1.6666666666666667
Canada,productD,0.1
India,productB,0.1
USA,productA,2.0
UK,productC,0.14285714285714285


In [0]:
%sql
describe detail silver_tb

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,ab593bea-330b-4827-8f97-a769a5ba6dbf,spark_catalog.default.silver_tb,null,dbfs:/user/hive/warehouse/silver_tb,2025-03-10T07:32:02.643Z,2025-03-10T07:37:15Z,List(),List(),4,5965,Map(delta.enableChangeDataFeed -> true),1,4,"List(appendOnly, changeDataFeed, checkConstraints, generatedColumns, invariants)",Map()


In [0]:
%sql
ALTER table silver_tb SET TBLPROPERTIES
(delta.enableChangeDataFeed=true);

In [0]:
new_products=[("Australia","productE",300,30)]
spark.createDataFrame(data=new_products,schema=columns).write.format("delta").mode("append").saveAsTable("silver_tb")

In [0]:
%sql
update silver_tb set stock='1200' where country='USA' and product='productA'

num_affected_rows
1


In [0]:
%sql
delete from silver_tb where country='UK' and product='productC'

num_affected_rows
1


In [0]:
%sql
describe history silver_tb

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2025-03-10T07:37:15Z,4991788245327114,pradeep114843@gmail.com,DELETE,"Map(predicate -> [""((country#3483 = UK) AND (product#3484 = productC))""])",null,List(2217622794786335),0310-072322-1v3eerc5,3,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1411, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 1, executionTimeMs -> 3619, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 2349, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1270)",null,Databricks-Runtime/15.4.x-scala2.12
3,2025-03-10T07:36:26Z,4991788245327114,pradeep114843@gmail.com,UPDATE,"Map(predicate -> [""((country#2905 = USA) AND (product#2906 = productA))""])",null,List(2217622794786335),0310-072322-1v3eerc5,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1418, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 1, executionTimeMs -> 5014, numDeletionVectorsUpdated -> 0, scanTimeMs -> 2155, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1635, rewriteTimeMs -> 2835)",null,Databricks-Runtime/15.4.x-scala2.12
2,2025-03-10T07:34:25Z,4991788245327114,pradeep114843@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2217622794786335),0310-072322-1v3eerc5,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1459)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-03-10T07:34:13Z,4991788245327114,pradeep114843@gmail.com,SET TBLPROPERTIES,"Map(properties -> {""delta.enableChangeDataFeed"":""true""})",null,List(2217622794786335),0310-072322-1v3eerc5,0,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12
0,2025-03-10T07:32:29Z,4991788245327114,pradeep114843@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(2217622794786335),0310-072322-1v3eerc5,null,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 4, numOutputBytes -> 5700)",null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%sql
select * from table_changes('silver_tb',1) order by _commit_timestamp

country,product,stock,sales,_change_type,_commit_version,_commit_timestamp
Australia,productE,300,30,insert,2,2025-03-10T07:34:25Z
USA,productA,1000,2000,update_preimage,3,2025-03-10T07:36:26Z
USA,productA,1200,2000,update_postimage,3,2025-03-10T07:36:26Z
UK,productC,700,100,delete,4,2025-03-10T07:37:15Z


In [0]:
change_df=spark.read.format('delta').option('readChangeData',True).option("startingVersion",2).table('silver_tb')
display(change_df)

country,product,stock,sales,_change_type,_commit_version,_commit_timestamp
USA,productA,1000,2000,update_preimage,3,2025-03-10T07:36:26Z
USA,productA,1200,2000,update_postimage,3,2025-03-10T07:36:26Z
UK,productC,700,100,delete,4,2025-03-10T07:37:15Z
Australia,productE,300,30,insert,2,2025-03-10T07:34:25Z


In [0]:
from pyspark.sql.functions import *
from delta.tables import *
delta_target=DeltaTable.forName(spark,'Gold_tb')

In [0]:
delta_source=spark.table('silver_versiontb')
display(delta_source)

country,product,stock,sales,_change_type,_commit_version,_commit_timestamp,rank
Australia,productE,300,30,insert,2,2025-03-10T07:34:25Z,1
UK,productC,700,100,delete,4,2025-03-10T07:37:15Z,1
USA,productA,1200,2000,update_postimage,3,2025-03-10T07:36:26Z,1


In [0]:
%sql
create or replace temporary view silver_versiontb as
select * from 
(select *,RANK() over(partition by country,product order by _commit_version desc) As rank from table_changes('silver_tb',2,5 )
where _change_type!='update_preimage')
where rank=1

In [0]:
%sql
select * from silver_versiontb

country,product,stock,sales,_change_type,_commit_version,_commit_timestamp,rank
Australia,productE,300,30,insert,2,2025-03-10T07:34:25Z,1
UK,productC,700,100,delete,4,2025-03-10T07:37:15Z,1
USA,productA,1200,2000,update_postimage,3,2025-03-10T07:36:26Z,1


In [0]:
delta_target.alias("target").merge(source=delta_source.alias('source'),
                                   condition="target.country=source.country and _change_type='udpate_postimage'").whenMatchedUpdate(\
                                       set={
                                           "SalesRate":"sales/stock" }).whenNotMatchedInsert(\
                                               values={
                                                   "country":"source.country",
                                                   "product":"source.product",
                                                   "SalesRate":"source.sales/source.stock"
                                                   }).execute()